In [ ]:
import pandas as pd

import os

import numpy as np

import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import seaborn as sns
from digitalhistopathology.embeddings.gene_embedding import GeneEmbedding

import anndata as ad
import glob
import scanpy as sc

from digitalhistopathology.datasets.real_datasets import HER2Dataset
import gzip
import pickle

import warnings
warnings.filterwarnings("ignore")

import matplotlib
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

In [ ]:
emb_raw_gene = ad.read_h5ad("../results/molecular/filtered_normalized_gene_expression.h5ad")


labels = pd.read_csv("../results/benchmark/her2_final_without_A/uni_full_models/invasive_cancer_clustering/kmeans/uni_full_koleo_16384_prototypes/invasive_labels_5_clusters_umap_min_dist_0.001_n_neighbors_250.csv", index_col=0)
palette={0: 'orange', 2:'gray', 4: 'lime', 3: 'magenta', 1: 'cyan', 'not invasive': 'white'}

with gzip.open('../results/compute_patches/her2_final_without_A/patches_info.pkl.gz') as f:
    patches_info = pickle.load(f)
    
    
def create_legend_gene_expression(gene_exp, mean, std):
    if gene_exp == 0:
        return "Not expressed"
    elif gene_exp < mean - std:
        return "Low expression"
    elif gene_exp < mean + std:
        return "Medium expression"
    else:
        return "High expression"

In [ ]:
patient ='E1'
gene_name = "HNRNPU"

In [ ]:
gene_names = ["ENY2", "TGFB1I1"]

for gene_name in gene_names:
    for patient in ["B1", "C1", "D1", "E1", "F1", "G2", "H1"]:


        print(patient)
        #print(emb_raw_gene[emb_raw_gene.obs["name_origin"] == patient].shape)
        subset_emb = GeneEmbedding()
        subset_emb.emb = emb_raw_gene[emb_raw_gene.obs["name_origin"] == patient]
        subset_emb.emb.obs = subset_emb.emb.obs.drop(['tumor', 'name_origin'], axis=1).merge(pd.DataFrame(patches_info), left_index=True, right_on='name')
        subset_emb.emb.obs.set_index('name', inplace=True)
        subset_emb.emb.obs['predicted_label'] = [labels.loc[idx, 'predicted_label'] if idx in labels.index else "not invasive" for idx in subset_emb.emb.obs.index]
        subset_emb.emb.obs[gene_name] = subset_emb.emb.X[:, subset_emb.emb.var.index.get_loc(gene_name)]
        mean = np.mean(subset_emb.emb.obs[gene_name])
        std = np.std(subset_emb.emb.obs[gene_name])
        # subset_emb.emb.obs[gene_name] = subset_emb.emb.obs.apply(lambda row: 'not invasive' if row['predicted_label'] == 'not invasive' else row[gene_name], axis=1)
        subset_emb.emb.obs[gene_name] = subset_emb.emb.obs[gene_name].apply(lambda x: create_legend_gene_expression(x, mean, std))
        subset_emb.plot_spot_location_with_color_on_origin_image(color=gene_name,
                                                                    palette={"Not expressed": 'white', "Low expression": 'yellow', "Medium expression": 'orange', "High expression": 'red'})
        plt.savefig(f"../Figures/Fig4/slide_{patient}_{gene_name}_expression_new.pdf", bbox_inches='tight')